In [1]:
from AnchorToConcrete import *

g:\Drive'ım\AnchorDesign\AnchorToConcrete.py:959: SyntaxWarning: invalid escape sequence '\ '
  """


# EXAMPLE 4.7-9— Base Connection for Bending

An exposed base connection is designed in this example that considers a base subjected to only flexural forces producing compression
at the toe of the base plate and tension in the anchor rods.

### Given:
A base connection of a W18×76 column has a moment from nonseismic forces as illustrated in Figure 4-31. Determine the
anchorage and base plate requirements. The concrete compressive strength, ƒc′, is 4,000 psi, and there are no adjacent edges. The
column is ASTM A992/A992M, and the plate is ASTM A572/A572M Grade 50 material.

### Solution:
From AISC Manual Tables 2-4 and 2-5, the material properties are as follows:

#### Column
ASTM A992/A992M
Fy = 50 ksi
Fu = 65 ksi

#### Plate
ASTM A572/A572M Grade 50
Fy = 50 ksi
Fu = 65 ksi
From AISC Manual Table 1-1, the column properties are as follows:
W18×76
d = 18.2 in.
bf = 11.0 in.
tf = 0.680 in.

### VARIABLES

In [2]:
inc  = 25.4 #mm
psi  = 0.00689476 #N/mm^2
kips = 4448.22 #N
ksi  = 6.89476 #N/mm^2

In [3]:
# CONCRETE
f_c   = 4000 * psi

# BASE PLATE
Fpy = 50 * ksi
Fpu = 65 * ksi

# ANCHOR RODS
Fay = 36 * ksi
Fau = 58 * ksi
d_a = 1.0 * inc
Abrg  = 1.50 * inc**2   # Design Guide Table-4.2
Ab    = 0.785 * inc**2  # Design Guide Table-4.2
nt = 8                  # Design Guide Table-4.1 number of threads
AseN = 0.606 * inc**2   # Design Guide Table-4.1
h_ef  = 12* inc         # embedment depth rods
n = 2                   # number of rods in tension

# OTHER VARIABLES
Anc_x  = 6 * inc    #Moment direction
Anc_y  = 24  * inc
Cax1   = 10000 * inc
Cax2   = 10000 * inc
Cay1   = 10000 * inc
Cay2   = 10000 * inc
e_N    = 0.0
e_h   = 0.0 * inc


fi_breakout  = 0.7   # Because no supplementary reinforcement was specified, ACI 318, Table 17.5.3(b)
fi_pullout   = 0.7   # ACI 318, Table 17.5.3(c)
fi_steelshear = 0.65 # ACI 318, Table 17.5.3(a)
fi_pryoutshear = 0.7 # ACI 318, Table 17.5.3(c)
fi_steeltension = 0.75 # ACI 318, Table 17.5.3(a),


 # FORCES
Vu = 116 * kips
Nu = 47 * kips
# Standart delikler ve kesmede etkin olabilecek şekilde ankrajlar beton kenarına yakın olmadığından kesme kuvveti ankrajlar arasında eşit dağıldığı kabul edilir. Aynı şekilde çekme durumunda da bir eksantrisite olmadığı için çekme kuvvetlerinin ankrajlara eşit dağıldığı kabul edilir.

# ACI318-19 17.5.1.3.1 ankraj aralığı 3*h_ef ten küçük olduğu için tüm ankrajlar grup olarak hareket edileceği düşünülüyor ve n=8 kabul ediliyor.
Vua_i = Vu/n
Nua_i = Nu/n

### TENSION CHECKS

#### AVAILABLE STEEL STRENGTH IN TENSION OF THE ANCHOR RODS

In [4]:
Nsa = NominalSteelStrengthOfAnchorInTension(A_se=AseN, f_ya=Fay, f_uta=Fau)
print(f"Nsa = {Nsa/10**3}kN - ({round(Nsa/4448.22,3)}kips)")

Design_Nsa = fi_steeltension * Nsa
print(f"fi_d*Nsa = {round(Design_Nsa/10**3,3)}kN - ({round(Design_Nsa/4448.2,3)}kips)"),
print(f"Is single anchor rods in tension capacity enough? ==> {Nua_i < Design_Nsa} - {round(Nua_i/1000,3)}kN < {round(Design_Nsa/1000,3)}kN - ({round(Nua_i/4448.2,3)}kips < {round(Design_Nsa/4448.2,3)}kips)")

Nsa = 156.346155kN - (35.148kips)
fi_d*Nsa = 117.26kN - (26.361kips)
Is single anchor rods in tension capacity enough? ==> True - 104.533kN < 117.26kN - (23.5kips < 26.361kips)


#### CONCRETE BREAKOUT STRENGTH IN TENSION

In [5]:
h_ef = h_ef_Check(h_ef,Cax1,Cax2,Cay1,Cay2,Anc_x,Anc_y)
print(f"h_ef = {round(h_ef/10**3,3)}m - ({round(h_ef/(25.4),3)}in)")

h_ef = 0.305m - (12.0in)


In [6]:
ANc = A_Nc(Anc_x, Anc_y, h_ef, Cax1, Cax2, Cay1, Cay2, n, TensionRegion=1)
print(f"ANc = {round(ANc/10**6,3)}m2 - ({round(ANc/25.4**2,3)}in^2)")

ANc = 0.975m2 - (1512.0in^2)


In [7]:
ANco = A_Nco(h_ef)
print(f"ANco = {round(ANco/10**6,3)}m2 - ({round(ANco/25.4**2,3)}in^2)")

ANco = 0.836m2 - (1296.0in^2)


In [8]:
BreakoutTension = ConcreteBreakoutStrengthOfAnchorInTension()
    
Nb = BreakoutTension.BasicSingleAnchorBreakoutStrength(kc=16, lambda_a=1, f_c=f_c, h_ef=h_ef)
print(f"Nb = {round(Nb/10**3,3)}kN - ({round(Nb/4448.22,3)}kips)")

psi_ecN = BreakoutTension.Psi_ecN_Get(eN=e_N, h_ef=h_ef)
print(f"psi_ecN = {psi_ecN}")

psi_edN = BreakoutTension.Psi_edN_Get(Ca_min=Cax1, h_ef=h_ef)
print(f"psi_edN = {psi_edN}")

psi_cN = BreakoutTension.Psi_cN_Get(InstalledType=CastInAnchorType.HexHeadBoltWithWasher, IsConcCracked = None)
print(f"psi_cN = {psi_cN}")

psi_cpN = BreakoutTension.Psi_cpN_Get(Cac=Cax1 , Ca_min=Cax1 , h_ef=h_ef, InstalledType=CastInAnchorType.HexHeadBoltWithWasher)
print(f"psi_cpN = {psi_cpN}")

Ncbg = BreakoutTension.ForGroupAnchor(A_Nc=ANc, A_Nco=ANco, Psi_ecN=psi_ecN, Psi_edN=psi_edN, Psi_cN=psi_cN,Psi_cpN=psi_cpN, Nb=Nb)
print(f"Ncbg = {round(Ncbg/10**3,3)}kN - ({round(Ncbg/4448.2,3)}kips)")

Design_Ncbg = fi_breakout * Ncbg
print(f"fi_d*Ncbg = {round(Design_Ncbg/10**3,3)}kN - ({round(Design_Ncbg/4448.2,3)}kips)")

print(f"Is concrete breakout strength in tension capacity enough? ==> {Nu < Design_Ncbg} - {round(Nu/1000,3)}kN < {round(Design_Ncbg/1000,3)}kN - ({round(Nu/4448.2,3)}kips < {round(Design_Ncbg/4448.2,3)}kips)")

Nb = 282.735kN - (63.561kips)
psi_ecN = 1.0
psi_edN = 1.0
psi_cN = 1.0
psi_cpN = 1.0
Ncbg = 329.857kN - (74.155kips)
fi_d*Ncbg = 230.9kN - (51.909kips)
Is concrete breakout strength in tension capacity enough? ==> True - 209.066kN < 230.9kN - (47.0kips < 51.909kips)


#### CONCRETE PULLOUT STRENGTH IN TENSION

In [9]:
PulloutTension = PulloutStrengthInTension()
    
psi_cp = PulloutTension.Psi_cP_Get(IsConcCracked=None)
print(f"psi_cp = {psi_cp}")
    
Np     = PulloutTension.BasicSingleAnchorPulloutStrength(A_brg=Abrg, 
                                                             f_c=f_c, 
                                                             e_h=e_h, 
                                                             d_a=d_a, 
                                                             AnchorType= CastInAnchorType.HexHeadBoltWithWasher)
print(f"Np = {round(Np/10**3,3)}kN - ({round(Np/4448.2,3)}kips)")
    
Npn = PulloutTension.SingleAnchorNominalPulloutStrength(Psi_cP=psi_cp, Np=Np)
print(f"Npn = {round(Npn/10**3,3)}kN - ({round(Npn/4448.2,3)}kips)")

Design_Npn = fi_pullout * Npn
print(f"fi_d*Ncbg = {round(Design_Npn/10**3,3)}kN - ({round(Design_Npn/4448.2,3)}kips)")

print(f"Is concrete pullout strength in tension capacity enough? ==> {Nua_i < Design_Npn} - {Nua_i/1000}kN < {Design_Npn/1000}kN - ({round(Nua_i/4448.2,3)}kips < {round(Design_Npn/4448.2,3)}kips)")


psi_cp = 1.0
Np = 213.515kN - (48.0kips)
Npn = 213.515kN - (48.0kips)
fi_d*Ncbg = 149.46kN - (33.6kips)
Is concrete pullout strength in tension capacity enough? ==> True - 104.53317000000001kN < 149.46030469999997kN - (23.5kips < 33.6kips)


#### CONCRETE SIDE-FACE BLOWOUT STRENGTH IN TENSION

Because hef ≤ 2.5ca1, side-face blowout is not applicable.

In [10]:
sideFaceBlowout = ConcreteSideFaceBlowoutStrengthOfHeadedAnchorInTension()
Nsb = sideFaceBlowout.SingleHeadedAnchorSideFaceBlowoutStrength(h_ef=h_ef, C_a1=Cax1, C_a2=Cax2,A_brg=Abrg, lambda_a=1.0, f_c=f_c)
Nsbg = sideFaceBlowout.MultipleHeadedAnchorSideFaceBlowoutStrength(s= Anc_x, h_ef=h_ef, C_a1=Cax1, N_sb=Nsb)
fi_sideway = Table1753b(SupReinforcement=False, AncTypeInstall=True, WhichEffect=0, AnchorCategory=None)
Design_Nsbg = fi_sideway*Nsbg
Design_Nsbg

7.000000000000001e+49

### SHEAR CHECKS

#### AVAILABLE STEEL STRENGTH IN SHEAR

In [11]:
steelStrengthShear = SteelStrengthOfAnchorInShear()
    
Asev = steelStrengthShear.A_seV(d_a=d_a, n_t=nt)
print(f"A_seV = {round(Asev,3)}mm2 - ({round(Asev/25.4**2,3)}in^2)")

# Design guide örneğinde efektif ankrajın çekme alanı alınmış(DG01-Table-4.1 A_seN) ACI da önerilen formül nominal alana yakın sonuç veriyor.(ACI318-19 - R17.7.1.2)
Vsa = steelStrengthShear.NominalSteelStrengthCastInHeadedAndHookedBoltAnchorInShear(A_seV=1.41*25.4**2, f_ya=Fay, f_uta= Fau)
print(f"Vsa = {round(Vsa/10**3,3)}kN - ({round(Vsa/4448.2,3)}kips)")

Design_Vsa = fi_steelshear * Vsa
print(f"fi*Vsa = {round(Design_Vsa/10**3,3)}kN - ({round(Design_Vsa/4448.2,3)}kips)")
    
print(f"Is available steel strength in shear capacity enough? ==> {Vua_i < Design_Vsa} - {Vua_i/1000}kN < {Design_Vsa/1000}kN - ({round(Vua_i/4448.2,3)}kips < {round(Design_Vsa/4448.2,3)}kips)")

A_seV = 501.61mm2 - (0.777in^2)
Vsa = 174.612kN - (39.255kips)
fi*Vsa = 113.498kN - (25.515kips)
Is available steel strength in shear capacity enough? ==> False - 257.99676kN < 113.49802035kN - (58.0kips < 25.515kips)


#### CONCRETE BREAKOUT STRENGTH IN SHEAR

In [22]:
BreakoutShear = ConcreteBreakoutStrengthOfAnchorInShear()
print("Because there are no edges adjacent to the base connection, the available concrete breakout strength in shear is not applicable.")
ShearDirection=1 ; IsWeldCommonPlate=False ; ha = 10000*25.4
ConcCrack = False; Reinforcement = False; ReinforcementBarNo = 0; StirrupsSpaced = 0.; eV = 0.0 ; t_attachment=13

Ca1  = BreakoutShear.Get_Ca1(Anc_x = Anc_x, 
                             Anc_y = Anc_y, 
                             ha = ha, 
                             Ca_x1=Cax1, 
                             Ca_x2=Cax2, 
                             Ca_y1=Cay1, 
                             Ca_y2=Cay2, 
                             ShearDirection=ShearDirection, 
                             IsWeldCommonPlate=IsWeldCommonPlate)
Ca2 = Anc_y+Ca1
Avco = BreakoutShear.A_Vco(C_a1=Ca1)
Avc  = BreakoutShear.A_Vc(Anc_x = Anc_x, Anc_y = Anc_y, ha = ha, Ca_x1=Cax1, Ca_x2=Cax2, Ca_y1=Cay1, Ca_y2=Cay2, ShearDirection=ShearDirection, IsWeldCommonPlate=IsWeldCommonPlate)
print(f"{Ca1=:.2f} ; {Avco=:.2f} ; {Avc=:.2f}")
le = BreakoutShear.Get_le(AnchorType = CastInAnchorType.HexHeadBoltWithWasher, h_ef =h_ef, d_a=d_a)
Vb = BreakoutShear.V_b(AnchorType=CastInAnchorType.HexHeadBoltWithWasher, l_e=le, d_a=d_a, f_c=f_c, Ca1=Ca1, t_attachment=t_attachment, s_min=Anc_x, Ca2=Anc_y, h_ef=h_ef)
Psi_ecV = BreakoutShear.Psi_ecV(eV = eV , Ca1 = Ca1)
Psi_edV = BreakoutShear.Psi_edV(Ca1 = Ca1, Ca2 = Ca2)
Psi_cV  = BreakoutShear.Psi_cV(ConcCrack = ConcCrack, Reinforcement = Reinforcement, ReinforcementBarNo = ReinforcementBarNo, StirrupsSpaced = StirrupsSpaced)
Psi_hV  = BreakoutShear.Psi_hV(Ca1 = Ca1, h_a = ha)
Vcbg = BreakoutShear.ForGroupAnchor(A_Vc=Avc, A_Vco=Avco, Psi_ecV=Psi_ecV, Psi_edV=Psi_edV, Psi_cV=Psi_cV, Psi_hV=Psi_hV, V_b=Vb)
print(f"{le=:.2f} ; {Vb=:.2f} ; {Psi_ecV=:.2f} ; {Psi_edV=:.2f} ; {Psi_cV=:.2f} ; {Psi_hV=:.2f} ; {Vcbg=:.2f} ; ")

Because there are no edges adjacent to the base connection, the available concrete breakout strength in shear is not applicable.
Ca1=169333.33 ; Avco=129031999492.00 ; Avc=129070709091.92
le=203.20 ; Vb=3293409749.92 ; Psi_ecV=1.00 ; Psi_edV=0.90 ; Psi_cV=1.00 ; Psi_hV=1.00 ; Vcbg=2967329961.96 ; 


#### CONCRETE PRYOUT STRENGTH IN SHEAR

In [13]:
PryShear = ConcretePryoutStrengthOfAnchorInShear()

Ncpg = Ncbg
    
Vcpg = PryShear.GroupAnchorConcPryoutStrengthInShear(h_ef=h_ef, N_cpg=Ncpg)
print(f"Vcpg = {round(Vcpg/10**3,3)}kN - ({round(Vcpg/4448.2,3)}kips)")

Design_Vcpg = fi_pryoutshear * Vcpg
print(f"fi*Vsa = {round(Design_Vcpg/10**3,3)}kN - ({round(Design_Vcpg/4448.2,3)}kips)")
    
print(f"Is available pryout strength in shear capacity enough? ==> {Vu < Design_Vcpg} - {Vu/1000}kN < {Design_Vcpg/1000}kN - ({round(Vu/4448.2,3)}kips < {round(Design_Vcpg/4448.2,3)}kips)")

Vcpg = 659.714kN - (148.31kips)
fi*Vsa = 461.8kN - (103.817kips)
Is available pryout strength in shear capacity enough? ==> False - 515.99352kN < 461.80001559999994kN - (116.001kips < 103.817kips)


### TENSION-SHEAR INTERACTION

In [14]:
DesignNn = min(Design_Npn,Design_Nsa,Design_Ncbg,Design_Nsbg)
DesignVn = min(Design_Vsa,Design_Vcpg)
ratioTS = TensionShearInteractionStrength(Nua_i, Vua_i, DesignNn, DesignVn)
ratioTS

1.812

### RESULTS

In [15]:
from pandas import DataFrame

In [25]:
limitState = ["Steel Failure in Tension",
              "Concrete Breakout in Tension",
              "Concrete Pullout in Tension",
              "Concrete Side-Face Blowout in Tension",
              "Concrete Splitting in Tension",
              "Concrete Bond Failure in Tension",
              "Steel Failure in shear",          
              "Concrete Pryout in shear",        
              "Concrete Breakout in shear",      
              "Tension-Shear Interaction"]
DCapacity = [   round(Design_Nsa/1000 ,3),
                round(Design_Ncbg/1000,3),
                round(Design_Npn/1000 ,3),
                round(Design_Nsbg/1000,3),
                0,
                0,
                round(Design_Vsa/1000 ,3),
                round(Design_Vcpg/1000,3),
                0,
                ratioTS]
Demands = [ Nua_i/10**3  , 
            Nu/1000      , 
            Nua_i/1000   , 
            0            ,  
            0            ,  
            0            ,  
            Vua_i/1000   , 
            Vu/1000      , 
            0            ,  
            1.2          ]

Checks = []
for C,D in zip(DCapacity,Demands):
    if C<D:
        Checks.append("x")
    else:
        Checks.append("Ok")
dct = {"Limit_State" : limitState, "Design_Capacity(kN)" : DCapacity, "Demand(kN)" : Demands, "Checks" : Checks}
df = DataFrame(dct)
df


,Limit_State,Design_Capacity(kN),Demand(kN),Checks
0,Steel Failure in Tension,1.172600e+02,104.53317,Ok
1,Concrete Breakout in Tension,2.309000e+02,209.06634,Ok
2,Concrete Pullout in Tension,1.494600e+02,104.53317,Ok
3,Concrete Side-Face Blowout in Tension,7.000000e+46,0.00000,Ok
4,Concrete Splitting in Tension,0.000000e+00,0.00000,Ok
5,Concrete Bond Failure in Tension,0.000000e+00,0.00000,Ok
6,Steel Failure in shear,1.134980e+02,257.99676,x
7,Concrete Pryout in shear,4.618000e+02,515.99352,x
8,Concrete Breakout in shear,0.000000e+00,0.00000,Ok
9,Tension-Shear Interaction,1.812000e+00,1.20000,Ok


| State                                 | Design Capacity | Demand
| -----------                           | -----------     | -----------     |
| Steel Failure in Tension              | Text            | Title           |
| Concrete Pullout in Tension           | Text            | Title           |
| Concrete Breakout in Tension          | Text            | Title           |
| Concrete Splitting in Tension         | Text            | Title           |
| Concrete Side-Face Blowout in Tension | Text            | Title           |
| Concrete Bond Failure in Tension      | Text            | Title           |
| Steel Failure in shear                | Text            | Title           |
| Concrete Pryout in shear              | Text            | Title           |
| Concrete Breakout in shear            | Text            | Title           |
| Tension-Shear Interaction             | Text            | Title           |